In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directoy.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
print(train.shape)
train.head()

# Data Description
As is shown in the table above, the characteristics of the data are as follows:
1. The size of train data set is (595212, 59), which means there are 595212 samples for trainning and each sample has 57 features(not including 'id' and 'target').
2. Feature names contain postfix 'bin' to indicate binary features and 'cat' to indicate categorical features. Features without postfix mentioned above are either continuous or ordinal. 
3. Values of -1 indicate that the feature value was missing.
4. In order to deal with data better, we classify the features as 4 types as follow.


In [ ]:
cat_features = []
bin_features = []
continue_features = []
ordinal_features = []
for i in train.columns[2:]:
    if i.endswith('_bin'):
        bin_features.append(i)
    elif i.endswith('_cat'):
        cat_features.append(i)
    elif train[i].dtype == 'float64':
        continue_features.append(i)
    elif train[i].dtype == 'int64':
        ordinal_features.append(i)
print ('categorical features:')
print (cat_features)
print ('binary features:')
print (bin_features)
print ('continue features:')
print (continue_features)
print ('ordinal features:')
print (ordinal_features)

In [ ]:
print ('binary features distributions on target = 1')
fig, ax = plt.subplots(6,3,figsize=(15,30))
for i in range(len(bin_features)):
    cat_perc = train[[bin_features[i], 'target']].groupby([bin_features[i]],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax[int(i/3),i%3], x=bin_features[i], y='target', data=cat_perc, order=cat_perc[bin_features[i]])
    ax[int(i/3),i%3].set_ylabel('% target = 1')
    ax[int(i/3),i%3].set_xlabel(bin_features[i])
    #sns.distplot(train[cat_features[i]].groupby([cat_features[i]],as_index=False).sum(),ax=ax[int(i/2),i%2])
plt.show()

In [ ]:
train.drop(['ps_calc_15_bin','ps_calc_16_bin','ps_calc_17_bin','ps_calc_18_bin','ps_calc_19_bin','ps_calc_20_bin'],axis=1,inplace=True)

In [ ]:
print ('categorical features distributions on target = 1')
fig, ax = plt.subplots(7,2,figsize=(15,30))
for i in range(len(cat_features)):
    cat_perc = train[[cat_features[i], 'target']].groupby([cat_features[i]],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax[int(i/2),i%2], x=cat_features[i], y='target', data=cat_perc, order=cat_perc[cat_features[i]])
    ax[int(i/2),i%2].set_ylabel('% target = 1')
    ax[int(i/2),i%2].set_xlabel(cat_features[i])
    #sns.distplot(train[cat_features[i]].groupby([cat_features[i]],as_index=False).sum(),ax=ax[int(i/2),i%2])
plt.show()

'ps_ind_02_cat','ps_ind_04_cat','ps_in_05_cat','ps_car_01_cat','ps_car_07_cat','ps_car_09_cat', -1 great proportion
'ps_car_02_car' -1 low proportion 

In [ ]:
print ('ordinal features distributions on target = 1')
fig, ax = plt.subplots(8,2,figsize=(15,30))
for i in range(len(ordinal_features)):
    cat_perc = train[[ordinal_features[i], 'target']].groupby([ordinal_features[i]],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax[int(i/2),i%2], x=ordinal_features[i], y='target', data=cat_perc, order=cat_perc[ordinal_features[i]])
    ax[int(i/2),i%2].set_ylabel('% target = 1')
    ax[int(i/2),i%2].set_xlabel(ordinal_features[i])
    #sns.distplot(train[cat_features[i]].groupby([cat_features[i]],as_index=False).sum(),ax=ax[int(i/2),i%2])
plt.show()

In [ ]:
#train.drop(['ps_calc_04','ps_calc_09'],axis=1,inplace=True)

In [ ]:
print ('continuous features distributions')
fig, ax = plt.subplots(5,2,figsize=(15,30))
for i in range(len(continue_features)):
    sns.distplot(train[continue_features[i]],ax=ax[int(i/2),i%2])
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(train['target'])
plt.xlabel('Target')
print ('Percentage of target = 1 :',(train['target'].sum()/train.shape[0])*100,'%')

As we can see, the **target** value is **imbalanced** so that we need to be careful.

 ## (a) Missing Values
Because values of -1 indicate that the feature was missing from the observation, the distribution of missing values can be shown by counting numbers of '-1' of each column.

In [ ]:
miss_features = []
miss_perc = []
miss_cat = []
for f in train.columns:
    miss = train[train[f] == -1][f].count()
    if miss > 0:
        miss_features.append(f)
        if f.endswith('_cat'):
            miss_cat.append(f)
        miss_perc.append(miss/train.shape[0])
plt.figure(figsize=(10,8))
plt.bar(range(len(miss_features)),miss_perc)
plt.xticks(range(len(miss_features)),miss_features,rotation=25,fontsize=12)
plt.ylabel('Percentage of missing values',fontsize=18)
plt.title('Missing Values distributions',fontsize=18)

As we can see above, there are 13 features that contains missing values. Especially,  '**ps_reg_03**', '**ps_car_03_cat**', '**ps_car_05_cat**' and '**ps_car_14**' have many missing values to be handled.

From the figure above, we can see that the proportions of missing values for some features are large, so **keeping the missing values as a separate category value** would be better than replacing them by the mode. These features are as follow:
1. 'ps_ind_02_cat'
2. 'ps_ind_04_cat'
3. 'ps_ind_05_cat'
4. 'ps_car_01_cat'
5. 'ps_car_03_cat'
6. 'ps_car_05_cat'
7. 'ps_car_07_cat'
8. 'ps_car_09_cat'

Maybe the customers with a missing value appear to have a much higher (in some cases much lower) probability to ask for an insurance claim. 
## (b) Handling Missing Values
According to the features classification, we can handle features as follow:
1. '**ps_car_03_cat**' and  '**ps_car_05_cat**' have a large proportion of missing values, so at first we **remove** these two features.
2. '**ps_reg_03**', '**ps_car_12**', and '**ps_car_14**' are continuous values, so we replace the missing values by the **mean**.
3. Missing values of **other** features are replaced by the **mode**. 


# Data Visualization
In order to understand the data better, we use some figures to show the relationship between features and target.
## Categorical Features
### Relationships between Categorical features and the target.

In [ ]:
plt.figure(figsize=(16,12))
plt.title('Pearson Correlation of features', y=1.05, size=15)
sns.heatmap(train.corr(),cmap="YlGnBu")

(ps_ind_16_bin,ps_ind_17_bin)  -0.52 cor
(ps_ind_16_bin,ps_ind_18_bin)  -0.59 cor


From the correlation map of features, we can see that the majority of the features display zero or no correlation to one another. This is a good news. There are eight pairs features that might have strong linear correlation as follow:
1. (ps_ind_12_bin, ps_ind_14)
2. (ps_ind_13_bin, ps_ind_14)
3. (ps_car_04_cat, ps_car_12)
4. (ps_car_04_cat, ps_car_13)
5. (ps_reg_01, ps_reg_03)
6. (ps_reg_02, ps_reg_03)
7. (ps_car_12, ps_car_13)
8. (ps_car_13, ps_car_15)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
target = train['target']
train.drop('target',axis=1,inplace=True)
train.drop('id',axis=1,inplace=True)

In [ ]:
import random
import xgboost as xgb
from sklearn import *
def gini(y, pred):
    fpr, tpr, thr = metrics.roc_curve(y, pred, pos_label=1)
    g = 2 * metrics.auc(fpr, tpr) - 1
    return g

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)
random.seed( 3 )
x1, x2, y1, y2 = model_selection.train_test_split(train, target, test_size=0.3, random_state=6)
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
params = {'eta': 0.09, 'max_depth': 6, 'objective': 'binary:logistic', 'seed': 16, 'silent': True, 'colsample_bytree': 0.6}
model = xgb.train(params, xgb.DMatrix(x1, y1), 1000,  watchlist, feval=gini_xgb, maximize=True, verbose_eval=50, early_stopping_rounds=100)

In [ ]:
xgb.plot_importance(booster=model,max_num_features=15)

In [ ]:
rf = RandomForestClassifier(n_estimators=150, max_depth=8, min_samples_leaf=4, max_features=0.2, n_jobs=-1, random_state=0)
rf.fit(train, target)
#features = train.columns.values
print("----- Training Done -----")
importance = rf.feature_importances_
features = train.columns.values
plt.figure(figsize=(10,12))
importance = pd.DataFrame(data=importance,index=features,columns=['importance'])
importance.sort_values('importance', ascending=False, inplace=True)
# Bar plot
# Order the bars descending on target mean
sns.barplot(x='importance', y=importance.index, data=importance)
sns.despine(left=True, bottom=True)

In [ ]:
importance

In [ ]:
a_arr = [0.01,0.05,0.1]
d_arr = [1,10,100,1000]
result = []
for a in a_arr:
    temp = []
    for d in d_arr:
        f = (1.0-(1.0-a)**d)
        temp.append(f)
    result.append(temp)
        

In [ ]:
plt.figure(figsize=(10,8))
plt.title('the fraction f', y=1.05, size=15)
sns.heatmap(result,xticklabels=d_arr,yticklabels=a_arr,cmap="YlGnBu",annot=True)
plt.xlabel('d',fontsize=15)
plt.ylabel('epsilon / a',fontsize=15)